In [3]:
pip install peewee;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peewee: filename=peewee-3.16.2-cp310-cp310-linux_x86_64.whl size=845782 sha256=ba3d8cfc08d1685b2ebea947df9242a28f9c260534c73271532fdb546af45273
  Stored in directory: /root/.cache/pip/wheels/24/76/50/b0b3e96184430b9bd19ad31f79a171dcebe8f47cb367401f1a
Successfully built peewee


In [4]:
pip install netcdf4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.2 MB/s eta 0:00:00


In [5]:
%pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 103.3 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
import requests
import pandas as pd
import peewee
from peewee import *
from netCDF4 import Dataset, num2date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
from shapely.geometry import Polygon, Point
import geopandas as gpd
import folium
import plotly.graph_objects as go

In [ ]:
caminho_para_pasta = '/content/drive/MyDrive/ICD/Colab/Alexandre'
caminho_para_arquivo = '/chuvas_C_00935056.csv'

# O arquivo apresentou erros de separador e codificação.
# Os dados começam na linha 12
# Não usa coluna como índice
df = pd.read_csv(f'{caminho_para_pasta}{caminho_para_arquivo}', sep=';', encoding="ISO-8859-1", skiprows = 12, index_col=False, decimal=',')
#print(df.columns)
# Muda o tipo da coluna 'Data' para datetime (string --> datetime)
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice
df.set_index(['Data','NivelConsistencia'], inplace=True)

# Exclui colunas desnecessárias do dataframe
listColumnsDrop = []
for i in range(73):
  if i > 41:
    listColumnsDrop += [i]
listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10]
df = df.drop(df.columns[listColumnsDrop], axis=1)

# Avalia se existe uma mesma data com diferentes níveis de consistência
# Classificar o dataframe por data
df.sort_values("Data", inplace=True)

# Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
df = df.reset_index()

# apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice
df.set_index(['Data','NivelConsistencia'], inplace=True)

# transforma as colunas de chuva em linhas com as respectivas datas
# Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
# Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
# Correção explicada durante o processo:

# usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
# São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

# Fiz uma cópia da minha data em formato de texto
df['Data1']=df['Data'].astype(str)

# Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
df['Data1'] = df['Data1'].str[:-2]

# Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
# (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)

# praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

# Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
# aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

# Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
# Mas aqui também forma datas como '1989-02-31'
df['Data1'] = df['Data1'] + df['Dia1']

# A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
# vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

# Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
df.dropna(inplace=True, subset=['Data1'])

# reorganiza as colunas
df = df[['Data', 'NivelConsistencia', 'Chuva']]

# Classificar o dataframe por data
df.sort_values("Data", inplace=True)

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
df.set_index(['Data'], inplace=True)

<ipython-input-4-f0d44a2eea23>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values("Data", inplace=True)


In [ ]:
periodo = df.loc['1990-01-01':'1991-01-01']
periodo = periodo.dropna(subset=['Chuva'])
periodo

,NivelConsistencia,Chuva
Data,,
1990-01-01,2,3.3
1990-01-02,2,3.3
1990-01-03,2,0.0
1990-01-04,2,0.0
1990-01-05,2,2.2
...,...,...
1990-12-28,2,0.0
1990-12-29,2,0.0
1990-12-30,2,0.0


In [ ]:
# definindo a função que calcula a média amostral
def sample_mean(data):
    return np.mean(data)

# definindo a função que calcula o desvio padrão amostral
def sample_std(data):
    return np.std(data, ddof=1)

# obtendo os dados de chuva da coluna 'Chuva'
data = periodo["Chuva"].values

# definindo o número de amostras de bootstrap
n_bootstraps = 1000

# gerando amostras de bootstrap
bootstrap_samples = np.random.choice(data, size=(n_bootstraps, len(data)), replace=True)

# calculando a estatística de interesse (média e desvio padrão) para cada amostra bootstrap
bootstrap_means = np.apply_along_axis(sample_mean, axis=1, arr=bootstrap_samples)
bootstrap_stds = np.apply_along_axis(sample_std, axis=1, arr=bootstrap_samples)

# calculando o intervalo de confiança de 95% para a média e o desvio padrão
mean_lower_ci = np.percentile(bootstrap_means, 2.5)
mean_upper_ci = np.percentile(bootstrap_means, 97.5)
std_lower_ci = np.percentile(bootstrap_stds, 2.5)
std_upper_ci = np.percentile(bootstrap_stds, 97.5)

print("Intervalo de confiança (95%) da média amostral:", mean_lower_ci, "-", mean_upper_ci)
print("Intervalo de confiança (95%) do desvio padrão amostral:", std_lower_ci, "-", std_upper_ci)

Intervalo de confiança (95%) da média amostral: 3.125833333333333 - 4.862862021857922
Intervalo de confiança (95%) do desvio padrão amostral: 7.076335064256208 - 10.599849135305467


In [ ]:
def prepararDF(df, codigo):
  # Muda o tipo da coluna 'Data' para datetime (string --> datetime)
  df['Data'] = df['DataHora'][:10]
  df['Data'] = pd.to_datetime(df['Data'], format='%Y/%m/%d')

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # Exclui colunas desnecessárias do dataframe
  listColumnsDrop = []
  for i in range(77):
    if i > 44:
      listColumnsDrop += [i]
  listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
  df = df.drop(df.columns[listColumnsDrop], axis=1)
  
  # Avalia se existe uma mesma data com diferentes níveis de consistência
  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
  df = df.reset_index()

  # apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
  df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # transforma as colunas de chuva em linhas com as respectivas datas
  # Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
  # Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
  # Correção explicada durante o processo:

  # usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
  # São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
  df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

  # Fiz uma cópia da minha data em formato de texto
  df['Data1']=df['Data'].astype(str)

  # Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
  df['Data1'] = df['Data1'].str[:-2]

  # Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
  # (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
  df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)
  
  # praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
  df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

  # Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
  # aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
  df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

  # Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
  # Mas aqui também forma datas como '1989-02-31'
  df['Data1'] = df['Data1'] + df['Dia1']

  # A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
  # vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
  df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

  # Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
  df.dropna(inplace=True, subset=['Data1'])

  # reorganiza as colunas
  df = df[['Data', 'NivelConsistencia', 'Chuva']]

  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
  #df.set_index(['Data'], inplace=True)
  df['Codigo'] = codigo
  return df

In [ ]:
# API para solicitar dados de estações sem telemetria
estacoes = ['936003','936004','936014','936045']
frames = []
for estacao in estacoes:
  url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
  params = {
      'codEstacao': estacao,
      'dataInicio': '',
      'dataFim': '',
      'tipoDados': '2',
      'nivelConsistencia': '',
  }
  try:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
  except ValueError:
    pass
  else:
    response = requests.get(url, params=params)
    xml = response.text
    df1 = pd.read_xml(xml,
                      xpath="//SerieHistorica",
                      namespaces={"xs": "http://www.w3.org/2001/XMLSchema"})
    df1 = prepararDF(df1, estacao)
    df1 = df1.dropna(subset=['Chuva'])
    #df1.reset_index()
    frames += [df1]
print(f'Foram carregadas {len(frames)} estações')

Foram carregadas 4 estações


<ipython-input-7-d8c20322755e>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values("Data", inplace=True)
<ipython-input-7-d8c20322755e>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Codigo'] = codigo


In [ ]:
df_936003 = frames[0]
df_936004 = frames[1]
df_936014 = frames[2]
df_936045 = frames[3]

Banco de dados

In [ ]:
db = peewee.SqliteDatabase('teste2.db')

class BaseModel(peewee.Model):
    
    class Meta:
        database = db

class Data(BaseModel):

    data = peewee.DateField(unique=True)
db.create_tables([Data])

class Estacao(BaseModel):
    nivel_consistencia = IntegerField()
    chuva = FloatField()
    codigo_estacao = peewee.CharField()

    # Chave estrangeira para a tabela Data
    date = peewee.ForeignKeyField(Data)
db.create_tables([Estacao])

In [ ]:
if __name__ == '__main__':
    try:
        Data.create_table()
        print("Tabela 'Data' criada com sucesso!")
    except peewee.OperationalError:
        print("Tabela 'Data' ja existe!")

    try:
        Estacao.create_table()
        print("Tabela 'Estacao' criada com sucesso!")
    except peewee.OperationalError:
        print("Tabela 'Estacao' ja existe!")

Tabela 'Data' criada com sucesso!
Tabela 'Estacao' criada com sucesso!


In [ ]:
# iterando pelos dataframes
for df in [df_936003, df_936004, df_936014]:
    for index, row in df.iterrows():
        # criando o objeto Data (se ainda não existe)
        data, created = Data.get_or_create(data=row['Data'])
        
        # criando o objeto Estacao com as informações da linha atual do dataframe
        Estacao.create(nivel_consistencia=row['NivelConsistencia'], chuva=row['Chuva'], codigo_estacao=row['Codigo'], date=data)

In [ ]:
dados = Estacao.select().join(Data).where(Data.data=='1989-01-02')

# Exibe a quantidade de registros que corresponde a nossa pesquisa
print(f'{dados.count()} estações')

for dado in dados:
    print(dado.chuva, dado.codigo_estacao, dado.nivel_consistencia)

3 estações
0.0 936003 1
19.0 936004 1
0.0 936014 1


In [ ]:
df_936004.loc[df_936004['Data']=='1989-01-02']

,Data,NivelConsistencia,Chuva,Codigo
13,1989-01-02,1,19.0,936004


In [ ]:
df_936045_dez = df_936045.set_index("Data").loc['2000-12-01':'2000-12-31']
#periodo = periodo.reset_index("Data")
df_936045_dez

,NivelConsistencia,Chuva,Codigo
Data,,,
2000-12-01,1,0.0,936045
2000-12-02,1,0.0,936045
2000-12-03,1,0.0,936045
2000-12-04,1,0.0,936045
2000-12-05,1,0.0,936045
2000-12-06,1,0.0,936045
2000-12-07,1,0.0,936045
2000-12-08,1,0.0,936045
2000-12-09,1,0.0,936045


Missão GPM

In [ ]:
# Diretório dos arquivos .nc4
diretorio = '/content/drive/MyDrive/ICD/Colab/Alexandre/GPM/'

# Coordenadas desejadas (X, Y)
lat = -9.1667
lon = -36.2167
	
# Criação do dataframe
df = pd.DataFrame(columns=['Date', 'Value'])

# Percorre os arquivos .nc4 no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.nc4'):
        # Caminho completo do arquivo
        caminho_arquivo = os.path.join(diretorio, arquivo)
        
        # Abre o arquivo NetCDF e carrega os dados
        ds = xr.open_dataset(caminho_arquivo)
        f1 = Dataset(caminho_arquivo)
        time = f1.variables["time"]
        ## Data for only one day
        dates = num2date(time[:], time.units)
        data = dates[0].strftime("%Y-%m-%d")
        # Obtém as datas dos dados
        #dates = num2date(ds.time[:], ds.time.units)
        
        # Converte a primeira data para o formato "YYYY-MM-DD"
        #data = dates[0].strftime('%Y-%m-%d')
        
        # Seleciona o valor de precipitação na coordenada (X, Y)
        valor = float(ds['HQprecipitation'].sel(lat=lat, lon=lon, method='nearest').values)
        
        # Adiciona a data e o valor ao dataframe
        df = df.append({'Date': data, 'Value': valor}, ignore_index=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
# Imprime o dataframe resultante
#print(df)
df_nc4 = df

<ipython-input-37-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-37-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-37-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-37-df11c0f116f8>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Date': data, 'Value': valor}, ignore_index=True)
<ipython-input-37-df11c0f116f8>:34: FutureWarning: T

In [10]:
from shapely.geometry import Point, box
# Coordenadas desejadas (X, Y)
lat = -9.35
lon = -35.84

# Calcula a metade da resolução (15 km)
half_resolution = 0.15

# Calcula as coordenadas para formar o quadrado
lon_min = lon - half_resolution
lon_max = lon + half_resolution
lat_min = lat - half_resolution
lat_max = lat + half_resolution

# Cria um quadrado com base nas coordenadas
square = box(lon_min, lat_min, lon_max, lat_max)

# Cria um ponto com base nas coordenadas desejadas
point = Point(lon, lat)

# Cria um GeoDataFrame com o quadrado e o ponto
gdf = gpd.GeoDataFrame(geometry=[square, point])

# Calcula o centro do quadrado
center_lat = square.centroid.y
center_lon = square.centroid.x

# Cria o mapa folium centrado no centro do quadrado
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Adiciona o quadrado ao mapa folium
folium.GeoJson(gdf.geometry[0]).add_to(m)

# Adiciona o ponto ao mapa folium
folium.Marker(location=[lat, lon]).add_to(m)

# Exibe o mapa folium
m

BOOTSTRAP df_936045_dez

In [ ]:
# definindo a função que calcula a média amostral
def sample_mean(data):
    return np.mean(data)

# definindo a função que calcula o desvio padrão amostral
def sample_std(data):
    return np.std(data, ddof=1)

# obtendo os dados de chuva da coluna 'Chuva'
data = df_936045_dez["Chuva"].values

# definindo o número de amostras de bootstrap
n_bootstraps = 1000

# gerando amostras de bootstrap
bootstrap_samples = np.random.choice(data, size=(n_bootstraps, len(data)), replace=True)

# calculando a estatística de interesse (média e desvio padrão) para cada amostra bootstrap
bootstrap_means = np.apply_along_axis(sample_mean, axis=1, arr=bootstrap_samples)
bootstrap_stds = np.apply_along_axis(sample_std, axis=1, arr=bootstrap_samples)

# calculando o intervalo de confiança de 95% para a média e o desvio padrão
mean_lower_ci = np.percentile(bootstrap_means, 2.5)
mean_upper_ci = np.percentile(bootstrap_means, 97.5)
std_lower_ci = np.percentile(bootstrap_stds, 2.5)
std_upper_ci = np.percentile(bootstrap_stds, 97.5)

print("Intervalo de confiança (95%) da média amostral:", mean_lower_ci, "-", mean_upper_ci)
print("Intervalo de confiança (95%) do desvio padrão amostral:", std_lower_ci, "-", std_upper_ci)

Intervalo de confiança (95%) da média amostral: 0.0 - 6.238709677419355
Intervalo de confiança (95%) do desvio padrão amostral: 0.0 - 16.444954123652405


Bootstrap DF_NC4

In [ ]:
# definindo a função que calcula a média amostral
def sample_mean(data):
    return np.mean(data)

# definindo a função que calcula o desvio padrão amostral
def sample_std(data):
    return np.std(data, ddof=1)

# obtendo os dados de chuva da coluna 'Chuva'
data = df_nc4["Value"].values

# definindo o número de amostras de bootstrap
n_bootstraps = 1000

# gerando amostras de bootstrap
bootstrap_samples = np.random.choice(data, size=(n_bootstraps, len(data)), replace=True)

# calculando a estatística de interesse (média e desvio padrão) para cada amostra bootstrap
bootstrap_means = np.apply_along_axis(sample_mean, axis=1, arr=bootstrap_samples)
bootstrap_stds = np.apply_along_axis(sample_std, axis=1, arr=bootstrap_samples)

# calculando o intervalo de confiança de 95% para a média e o desvio padrão
mean_lower_ci = np.percentile(bootstrap_means, 2.5)
mean_upper_ci = np.percentile(bootstrap_means, 97.5)
std_lower_ci = np.percentile(bootstrap_stds, 2.5)
std_upper_ci = np.percentile(bootstrap_stds, 97.5)

print("Intervalo de confiança (95%) da média amostral:", mean_lower_ci, "-", mean_upper_ci)
print("Intervalo de confiança (95%) do desvio padrão amostral:", std_lower_ci, "-", std_upper_ci)

Intervalo de confiança (95%) da média amostral: 0.02645161267249815 - 1.0682700540750254
Intervalo de confiança (95%) do desvio padrão amostral: 0.08245396185545653 - 2.3617537920874434


In [ ]:
df_936045_dez

,NivelConsistencia,Chuva,Codigo
Data,,,
2000-12-01,1,0.0,936045
2000-12-02,1,0.0,936045
2000-12-03,1,0.0,936045
2000-12-04,1,0.0,936045
2000-12-05,1,0.0,936045
2000-12-06,1,0.0,936045
2000-12-07,1,0.0,936045
2000-12-08,1,0.0,936045
2000-12-09,1,0.0,936045


Comparação entre dados de chuva por satélite e estação

In [ ]:

# Unir os dois dataframes com base nas datas
merged_df = df_nc4.merge(df_936045_dez, left_on='Date', right_on='Data')

# Definir as datas como índice
merged_df.set_index('Date', inplace=True)

# Criar figura interativa
fig = go.Figure()

# Adicionar barras do DataFrame df_nc4
fig.add_trace(go.Bar(
    x=merged_df.index,
    y=merged_df['Value'],
    name='df_nc4'
))

# Adicionar barras do DataFrame df_936045_dez
fig.add_trace(go.Bar(
    x=merged_df.index,
    y=merged_df['Chuva'],
    name='df_936045_dez'
))

# Atualizar layout do gráfico
fig.update_layout(
    title='Valores de Chuva, Dez 2000',
    #xaxis_tickangle=-45,
    xaxis=dict(
        title='Data'
    ),
    yaxis=dict(
        title='Valor de Chuva'
    )
)

# Exibir gráfico interativo
fig.show()